# Mediator

A mediator is an agent that mediates DIDComm messages on behalf of other agents. Specifically Alice can request this agent act as a mediator for their messages, then when creating an invitation, she specifies the mediators endpoint as the place for messages to be sent and adds the mediators keys for routing of these messages. Anyone wishing to send Alice a DIDComm message must first encrypt under Alice's key then again under the Mediators key before sending the message to the endpoint specified by Alice. The mediators endpoint.

### Useful Material

* [Aries-RFC-046 Mediators and Relays](https://github.com/hyperledger/aries-rfcs/tree/master/concepts/0046-mediators-and-relays)
* [Aries-RFC-094 Cross Domain Messaging](https://github.com/hyperledger/aries-rfcs/tree/master/concepts/0094-cross-domain-messaging)
* [DIDComm Messaging Specification - Routing](https://identity.foundation/didcomm-messaging/spec/#routing)

### 1. Pull in dependencies

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 2. Instatiate the controller for our Agent

The arguments depend on how the aca-py agent was initiated. See the manage and docker-compose.yml files for more details.

In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8042
WEBHOOK_BASE = ""
ADMIN_URL = "http://mediator-agent:8041"


In [3]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, mediation=True)

### 3. Listen for webhooks and register default listeners

TODO: Implement mediator webhooks. These currently need to be implemented in ACA-Py first. We could even help do this. See issue - https://github.com/hyperledger/aries-cloudagent-python/issues/950 

In [4]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.webhook_listener.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


### 4. Use the controller to create an invitation from our agent

An invitation is a JSON object, as shown below, it contains the relevant information required for another agent to connect with it and exchange identifiers. This object must be passed to the agent Alice wishes to connect to out of band, in this instance we will just copy it across.

In [5]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(multi_use="true")
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection ID a347eb87-8018-4d3e-a051-a945fcf52ff1
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'e08a7e80-d96b-43ba-9ea8-368e5e61eeb6', 'serviceEndpoint': 'https://7eceb3b5d5a4.ngrok.io', 'label': 'MEDIATOR', 'recipientKeys': ['69hmbswFgse281vL3Z7eAzv8mc6m5RwCYuTgZxhGk4GU']}
wallet None
Handle Webhook - connections {'connection_id': 'a347eb87-8018-4d3e-a051-a945fcf52ff1', 'accept': 'auto', 'invitation_key': '69hmbswFgse281vL3Z7eAzv8mc6m5RwCYuTgZxhGk4GU', 'created_at': '2021-03-31 17:42:19.462303Z', 'invitation_mode': 'multi', 'their_role': 'invitee', 'routing_state': 'none', 'state': 'invitation', 'updated_at': '2021-03-31 17:42:19.462303Z', 'rfc23_state': 'invitation-sent'}
Connection Handler Called
Connection a347eb87-8018-4d3e-a051-a945fcf52ff1 in State invitation
wallet None
Handle Webhook - connections {'connection_id': '49431565-9d12-42e9-8341-8699834fc785', 'accept': 'auto', 'invitation_key': '69hmbswFgse281vL3Z7eAzv8mc6m5RwCYuTgZ

### Copy the invitation output to any agent that you needs to establish a mediator. For example where you came from - [Alice](http://localhost:8888/lab/tree/Alice/Part%202%20-%20Mediation%20of%20communication%20-%20Alice.ipynb)

**Example** invitation you're supposed to copy:

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'fee0f080-88c2-49b1-9869-9117524f010c', 'label': 'MEDIATOR', 'serviceEndpoint': 'https://f6d0c3340147.ngrok.io', 'recipientKeys': ['53gFmaMTJXVy5xBKJKZeRJYs6dL7SNG55gR1W6WuAvpN']}

### Check Mediation Records

If you do this before accepting the invitation in Alice's notebook, this should be empty.

You can return here later (after accepting) and will find that this then shows the mediated conneciton we have established,

In [6]:
response = await agent_controller.mediation.get_mediation_records()

for record in response:
    print("Mediation Record")
    print("connection_id", record["connection_id"])
    print("State", record["state"])

Mediation Record
connection_id 49431565-9d12-42e9-8341-8699834fc785
State request


## Terminate Meditator Controller

**Only do this when you have completed the full tutorial**

In [ ]:
response = await agent_controller.terminate()
print(response)